In [32]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")
 
if module_path not in sys.path:
    sys.path.append(module_path)

In [35]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / 'batch' / 'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)


In [36]:
df_imgs

,Condition,FOV,Path
0,100nM,P10,Y:\coskun-lab\Thomas\23_PLA_revision\data\batc...
1,100nM,P21,Y:\coskun-lab\Thomas\23_PLA_revision\data\batc...
2,Control,P10,Y:\coskun-lab\Thomas\23_PLA_revision\data\batc...
3,Control,P21,Y:\coskun-lab\Thomas\23_PLA_revision\data\batc...


# Read PPI

In [37]:
PPI_save_path =  data_dir / 'batch' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

In [38]:
df.head()

,Cyto,x,y,PPI,Nuclei,Condition,FOV
0,0,2445,3266,Sox2/Oct4,0,100nM,P10
1,0,2445,3266,Sox2/Oct4,0,100nM,P10
2,0,2445,3266,Sox2/Oct4,0,100nM,P10
3,0,2445,3266,Sox2/Oct4,0,100nM,P10
4,313,3363,1184,Sox2/Oct4,0,100nM,P10


In [39]:
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']
df_cell = (df_cell.pivot_table(values='Count_cell', columns='PPI', index=['Condition', 'FOV','Id'])
.fillna(0)
.reset_index()
.melt(id_vars=['Condition', 'FOV','Id'], value_name='Count_cell', var_name='PPI')
)

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']
df_nuclei = (df_nuclei.pivot_table(values='Count_nuclei', columns='PPI', index=['Condition', 'FOV','Id'])
.fillna(0)
.reset_index()
.melt(id_vars=['Condition', 'FOV','Id'], value_name='Count_nuclei', var_name='PPI')
)

In [40]:
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all.loc[df_all['Count_cell'] < df_all['Count_nuclei'],'Count_cell'] = df_all.loc[df_all['Count_cell'] < df_all['Count_nuclei']]['Count_nuclei']


In [41]:
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei']

In [42]:
df_all.Count_cell.describe()

count    4236.000000
mean       10.051700
std        13.898635
min         0.000000
25%         2.000000
50%         6.000000
75%        13.000000
max       207.000000
Name: Count_cell, dtype: float64

In [43]:
# Filter out by maximum number of counts per cell
min_count = 0
max_count = 100

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Count_cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Count_cell'] < max_count).all())
df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe()

C:\Users\thu71\AppData\Local\Temp\ipykernel_16356\489516438.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe()


,Count_cell,Count_nuclei,Count_cyto
count,1049.000000,1049.0,1049.000000
mean,38.517636,0.0,38.517636
std,25.182766,0.0,25.182766
min,1.000000,0.0,1.000000
25%,21.000000,0.0,21.000000
50%,32.000000,0.0,32.000000
75%,51.000000,0.0,51.000000
max,162.000000,0.0,162.000000


In [44]:
df_all

,Condition,FOV,Id,PPI,Count_cell,Count_nuclei,Count_cyto
0,100nM,P10,8,Bim/Tom20,10.0,0,10.0
1,100nM,P10,9,Bim/Tom20,3.0,0,3.0
2,100nM,P10,17,Bim/Tom20,1.0,0,1.0
3,100nM,P10,18,Bim/Tom20,10.0,0,10.0
4,100nM,P10,19,Bim/Tom20,4.0,0,4.0
...,...,...,...,...,...,...,...
4231,Control,P21,191,p-P90rsk/NF-Kb,3.0,0,3.0
4232,Control,P21,192,p-P90rsk/NF-Kb,5.0,0,5.0
4233,Control,P21,194,p-P90rsk/NF-Kb,8.0,0,8.0
4234,Control,P21,195,p-P90rsk/NF-Kb,0.0,0,0.0


# ML model

In [45]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import wandb

In [46]:
condition = 'ML'
project_name = f'PLA_batch_{condition}'

In [47]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [48]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.loc[:, df_count.columns.get_level_values(0)=='Count_cell']
df_count_cell = pd.DataFrame(scaler.fit_transform(df_count_cell), index=df_count_cell.index, columns=df_count_cell.columns, )

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [49]:
df_count.index.get_level_values('FOV').unique()

Index(['P10', 'P21'], dtype='object', name='FOV')

In [50]:
## Train on P10 and evaluate on P21
train = 'P10'
held = 'P21'
# Run model on cell count
df_train = df_count_cell[df_count.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count.index.get_level_values('FOV') == held]
X = df_train.values
y = le.transform(df_train.index.get_level_values(0))
X_held = df_held.values
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

715 715 334 334


In [51]:
scaler = StandardScaler()


In [52]:
## Train on P10 and evaluate on P21
train = 'P10'
held = 'P21'

# Run model on cell count
df_train = df_count_cell[df_count_cell.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count_cell.index.get_level_values('FOV') == held]

X = df_train.values
# X = scaler.fit_transform(X)
y = le.transform(df_train.index.get_level_values(0))

X_held = df_held.values
# X_held = scaler.fit_transform(X_held)
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

# K fold training
kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)
        y_pred_held = model.predict(X_held)
        y_probas_held = model.predict_proba(X_held)

        run = wandb.init(project=project_name, group=model_name+'_'+train, name=model_name+f'_{train}_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        accuracy_held = metrics.accuracy_score(y_held, y_pred_held)
        f1 = metrics.f1_score(y_test, y_pred)
        f1_held = metrics.f1_score(y_held, y_pred_held)
        auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        auc_held = metrics.roc_auc_score(y_held, y_probas_held[:, 1])
        wandb.log({"accuracy": accuracy, 'accuracy_held': accuracy_held, 'f1':f1, 'f1_held': f1_held, 'auc': auc, 'auc_held': auc_held})
    run.finish()

715 715 334 334


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.70293
accuracy_held,0.51796
auc,0.6135
auc_held,0.49819
f1,0.81558


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.64706
accuracy_held,0.53593
auc,0.55976
auc_held,0.51242
f1,0.76536


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.63866
accuracy_held,0.52395
auc,0.53761
auc_held,0.56871
f1,0.76882


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.59414
accuracy_held,0.49102
auc,0.52496
auc_held,0.48378
f1,0.70871


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.70711
accuracy_held,0.52096
auc,0.56831
auc_held,0.55066
f1,0.82051


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.64286
accuracy_held,0.52096
auc,0.59188
auc_held,0.56092
f1,0.78036


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.63866
accuracy_held,0.51796
auc,0.57328
auc_held,0.56817
f1,0.77604


In [53]:
## Train on P10 and evaluate on P21
train = 'P21'
held = 'P10'

# Run model on cell count
df_train = df_count_cell[df_count_cell.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count_cell.index.get_level_values('FOV') == held]

X = df_train.values
# X = scaler.fit_transform(X)
y = le.transform(df_train.index.get_level_values(0))

X_held = df_held.values
# X_held = scaler.fit_transform(X_held)
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

# K fold training
kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)
        y_pred_held = model.predict(X_held)
        y_probas_held = model.predict_proba(X_held)

        run = wandb.init(project=project_name, group=model_name+'_'+train, name=model_name+f'_{train}_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        accuracy_held = metrics.accuracy_score(y_held, y_pred_held)
        f1 = metrics.f1_score(y_test, y_pred)
        f1_held = metrics.f1_score(y_held, y_pred_held)
        auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        auc_held = metrics.roc_auc_score(y_held, y_probas_held[:, 1])
        wandb.log({"accuracy": accuracy, 'accuracy_held': accuracy_held, 'f1':f1, 'f1_held': f1_held, 'auc': auc, 'auc_held': auc_held})
    run.finish()

334 334 715 715


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.57143
accuracy_held,0.56503
auc,0.572
auc_held,0.51903
f1,0.57143


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.4955
accuracy_held,0.52168
auc,0.49854
auc_held,0.52646
f1,0.51724


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.57658
accuracy_held,0.47273
auc,0.59821
auc_held,0.49872
f1,0.60504


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.55357
accuracy_held,0.54406
auc,0.54998
auc_held,0.51727
f1,0.5098


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.51786
accuracy_held,0.60979
auc,0.56718
auc_held,0.56939
f1,0.51786


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.54054
accuracy_held,0.58601
auc,0.55823
auc_held,0.57025
f1,0.52336


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.54955
accuracy_held,0.58881
auc,0.55721
auc_held,0.55651
f1,0.59677
